In [32]:
### LLM
from langchain_ollama import ChatOllama

local_llm = "mistral:7b-instruct"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

In [12]:
## Tavily web-search
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("TAVILY_API_KEY")
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [44]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Define allowed domains for Tunisian governmental financial institutions
tunisian_gov_domains = [
    "finances.gov.tn",        # Ministry of Finance
    "douane.gov.tn",          # Tunisian Customs
    "impots.finances.gov.tn", # Tax authority
    "cga.gov.tn",             # General Committee of Insurance
    "cmf.tn",                 # Financial Market Council
    "portail.finances.gov.tn" # Finance Ministry Portal
    "swiver.io"
]

# Create the search tool with domain filtering
web_search_tool = TavilySearchResults(
    k=3,
    include_domains=tunisian_gov_domains
)

# To ensure only 3 results are returned, you can modify how you use the results
def search_and_limit(query):
    results = web_search_tool.invoke(query)
    # Explicitly limit to first 3 results
    return results[:3]

In [45]:
### Generate
### Router
import json
from langchain_core.messages import HumanMessage, SystemMessage
# Prompt
rag_prompt = """Vous êtes un assistant pour des tâches de question-réponse.

Voici le contexte à utiliser pour répondre à la question :

{context}

Réfléchissez soigneusement au contexte ci-dessus.

Maintenant, examinez la question de l'utilisateur :

{question}

Fournissez une réponse à cette question en utilisant uniquement le contexte ci-dessus.

Utilisez un maximum de trois phrases et gardez la réponse concise.

Réponse :"""

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc['content'] for doc in docs)

In [47]:
query = "retenue à la source appliqué sur les honoraires en 2021?"
# Test
docs = web_search_tool.invoke(query)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=query)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

2025-05-09 13:13:04,723 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


 La retenue à la source sur les honoraires n'a pas changé en 2021, elle reste à 15%.
